In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from random import shuffle

import skimage.io as io

In [2]:
# Comparison between original image and resized image

# Read image and resize to (224, 224)
# cv2 load images as BGR, convert it to RGB
# addr = 'Temp/paper16.jpg'
# img = Image.open(addr)
# img.show()
# img = cv2.imread(addr)
# img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = img.astype(np.float32)
# imgplot = plt.imshow(img)

# Parameters

In [3]:
DATASET_PATH = "Dataset/**/*.jpg"
CLASS_LABELS = ['compost', 'landfill', 'recyclable']
TRAIN_VAL_TEST = {'train': 0.6, 'val': 0.8, 'test': 1} # Train: 60%, Val: 20%, Test: 20%
BATCH_SIZE = 5
IMAGE_RESIZE_SHAPE = (224, 224)
IMAGE_RESIZE_SHAPE_LIST = [224, 224, 3]
IMAGE_SIZE = 224
# NUM_CHANNELS = 3 # RGB
# NUM_CLASSES = 3 # Compost, Landfill & Recyclable
# NUM_EPOCHS = 5
# NUM_FILTERS = 32
# FILTER_SHAPE = [5, 5]
# POOL_SHAPE = [2, 2]

# Collect & Split Data

In [4]:
def collect_split_data():
    
    labels = []
    
    files = glob.glob(DATASET_PATH)
    for file in files:
        if CLASS_LABELS[0] in file:
            labels.append(0)
        elif CLASS_LABELS[1] in file:
            labels.append(1)
        elif CLASS_LABELS[2] in file:
            labels.append(2)
        else:
            print("Error: Image filename does not contain correct label.")

    c = list(zip(files, labels))
    shuffle(c)
    files, labels = zip(*c)
    
    train_img = files[0:int(TRAIN_VAL_TEST['train'] * len(files))]
    train_labels = labels[0:int(TRAIN_VAL_TEST['train'] * len(files))]
    val_img = files[int(TRAIN_VAL_TEST['train'] * len(files)) : int(TRAIN_VAL_TEST['val'] * len(files))]
    val_labels = labels[int(TRAIN_VAL_TEST['train'] * len(files)) : int(TRAIN_VAL_TEST['val'] * len(files))]
    test_img = files[int(TRAIN_VAL_TEST['val'] * len(files)):]
    test_labels = labels[int(TRAIN_VAL_TEST['val'] * len(files)):]
        
    return len(files), train_img, train_labels, val_img, val_labels, test_img, test_labels

# Write data to tfrecords

In [5]:
def load_image(addr):
    # Read, resize and convert to RGB (since cv2 loads images as BGR)
    img = Image.open(addr)
    img = cv2.imread(addr)
    img = cv2.resize(img, IMAGE_RESIZE_SHAPE, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def create_tfrecord(files, labels, train_val_test):
        
    # Open .tfrecords file
    writer = tf.python_io.TFRecordWriter(train_val_test+'.tfrecords')
    
    for i in range(len(files)):
    
        # Load image and its label
        img = load_image(files[i])
        label = labels[i]

        # Create a feature
        feature = { train_val_test+'/label': _int64_feature(label),
                    train_val_test+'/image': _bytes_feature(tf.compat.as_bytes(img.tobytes()))}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))

        # Serialize to string and write to file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

def create_tfrecords(train_img, train_labels, val_img, val_labels, test_img, test_labels):
    
    create_tfrecord(train_img, train_labels, 'train')
    create_tfrecord(val_img, val_labels, 'val')
    create_tfrecord(test_img, test_labels, 'test')

# Read data from tfrecords

In [6]:
def read_from_tfrecords(train_val_test):
    
    file = glob.glob(train_val_test+'.tfrecords')
    

    feature = { train_val_test+'/image': tf.FixedLenFeature([], tf.string),
                train_val_test+'/label': tf.FixedLenFeature([], tf.int64) }

    # Enqueue train.tfrecords
    filename_queue = tf.train.string_input_producer(file, num_epochs=1)

    # Define reader and read file from queue
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    
    # Convert serialized data back to arrays and numbers
    image = tf.decode_raw(features[train_val_test+'/image'], tf.float32)
    label = tf.cast(features[train_val_test+'/label'], tf.int32)
    
    # Reshape image data to original shape
    image = tf.reshape(image, IMAGE_RESIZE_SHAPE_LIST)
    
    # CHECK: Delete later
    image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size=2, capacity=10, 
                                                          num_threads=1, min_after_dequeue=0)
    
    return image_batch, label_batch

# Main

In [7]:
# Collect data from "Dataset/", shuffle, and split it into training, validation & testing set.
data_size, train_img, train_labels, val_img, val_labels, test_img, test_labels = collect_split_data()

# Creates train.tfrecords, val.tfrecords & test.tfrecords
create_tfrecords(train_img, train_labels, val_img, val_labels, test_img, test_labels)


images, labels = read_from_tfrecords('train')
with tf.Session() as sess:
        sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        
        for i in range(3):
            img, anno = sess.run([images, labels])
            print(img[0, :, :, :].shape)

            print('current batch')

            # We selected the batch size of two
            # So we should get two image pairs in each batch
            # Let's make sure it is random

            io.imshow(img[0, :, :, :])
            io.show()

            io.imshow(anno[0, :, :, 0])
            io.show()

            io.imshow(img[1, :, :, :])
            io.show()

            io.imshow(anno[1, :, :, 0])
            io.show()
        


(224, 224, 3)
current batch
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: shuffle_batch/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch/random_shuffle_queue, Reshape, Cast)]]


ValueError: Images of type float must be between -1 and 1.